In [19]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
import requests
import json
import pprint
import parsing_statements as ps

pp = pprint.PrettyPrinter(indent=4)

In [20]:
cik = "0000719955"
headers = {"User-Agent": "russ@sunriseanalysis.com"}

In [21]:
def get_xbrl_links(cik):
    # Define the URL and headers
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    headers = {"User-Agent": "russ@sunriseanalysis.com"}
    # Make the request
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Failed to get data. HTTP status code: {response.status_code}")
    # Extract data and filter for 10-K and 10-Q forms
    data = pd.DataFrame(data["filings"]["recent"])
    data10k = data[data["form"] == "10-K"]
    data10q = data[data["form"] == "10-Q"]

    # Function to construct the URL
    def construct_url(row):
        base_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/"
        accession_number = row["accessionNumber"].replace("-", "")
        primary_document = row["primaryDocument"] # replace(".", "_") + ".xml"
        return base_url + accession_number + "/" + primary_document

    # Construct the URLs for "10-K" and "10-Q" forms
    urls_10k = data10k.apply(construct_url, axis=1).tolist()
    urls_10q = data10q.apply(construct_url, axis=1).tolist()

    return urls_10k, urls_10q

In [22]:
url10k, url10q = get_xbrl_links(cik)

In [23]:
url10k

['https://www.sec.gov/Archives/edgar/data/0000719955/000162828023009175/wsm-20230129.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000162828022007494/wsm-20220130.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000119312521100319/d93631d10k.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000119312520088937/d856872d10k.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000119312519097973/d683578d10k.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000119312518102232/d689546d10k.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000119312517104341/d265187d10k.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000119312516525847/d120289d10k.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000119312515118009/d851953d10k.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000119312514129974/d659151d10k.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000119312513142309/d465401d10k.htm',
 'https:/

In [24]:
url10q

['https://www.sec.gov/Archives/edgar/data/0000719955/000162828023019968/wsm-20230430.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000162828022031309/wsm-20221030.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000162828022024296/wsm-20220731.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000162828022015963/wsm-20220501.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000162828021024590/wsm-20211031.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000162828021018335/wsm-20210801.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000071995521000007/wsm-20210502.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000119312520310847/d97689d10q.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000119312520242226/d87421d10q.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000119312520162400/d919457d10q.htm',
 'https://www.sec.gov/Archives/edgar/data/0000719955/000119312519312802/d813200d10q.htm',
 'htt

In [25]:
response = requests.get(url10q[10], headers=headers)
content = response.content
bal, state = ps.parse_xml_from_content(content)